# Practica 5

In [1]:
import cv2
from deepface import DeepFace
import numpy as np

# Configuración de captura de video y máscara
vid = cv2.VideoCapture(0)
mascara = cv2.imread("careta.png", cv2.IMREAD_UNCHANGED)
scaleFactor = 1

while True:      
    # Captura fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:  
        obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['gender'])

        # Comprobar si se detecta una cara con suficiente confianza
        if (obj[0]["face_confidence"] > 0.5):
            landmarks = obj[0]['region']
            x, y, w, h = landmarks["x"], landmarks["y"], landmarks["w"], landmarks["h"]
            
            # Obtener la posición de los ojos, puede ser que no detecte
            if (landmarks["left_eye"]):
                eye_left = landmarks["left_eye"]
                eye_left_x = eye_left[0]
                eye_left_y = eye_left[1]
            if (landmarks["right_eye"]):
                eye_right = landmarks["right_eye"]
                eye_right_x = eye_right[0]
                eye_right_y = eye_right[1]
            else:
                x, y, w, h = landmarks["x"], landmarks["y"], landmarks["w"], landmarks["h"]
            
                resized = cv2.resize(mascara, (int(w*scaleFactor), int(h*scaleFactor)))
                W, H = resized.shape[:2]
                
                overlay_rgb = resized[:, :, :3]  # Canales de color
                alpha_mask = resized[:, :, 3] / 255.0  # Normalizar canal alfa

                frame[y:y+H, x:x+W] = (1.0 - alpha_mask[..., None]) * frame[y:y+H, x:x+W] + alpha_mask[..., None] * overlay_rgb    
                continue
            
            # Calcular el ángulo de rotación con en la diferencia de altura de los ojos
            delta_x = eye_left_x - eye_right_x
            delta_y = eye_left_y - eye_right_y

            angle = -np.degrees(np.arctan2(delta_y, delta_x))

            # Redimensionar la máscara
            resized = cv2.resize(mascara, (int(w * scaleFactor), int(h * scaleFactor)))
            
            # Rotar la máscara según el ángulo calculado
            (h_masc, w_masc) = resized.shape[:2]
            center = (w_masc // 2, h_masc // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated_mask = cv2.warpAffine(resized, rotation_matrix, (w_masc, h_masc), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

            # Separar canales de color y alfa de la máscara rotada
            overlay_rgb = rotated_mask[:, :, :3]  # Canales de color
            alpha_mask = rotated_mask[:, :, 3] / 255.0  # Normalizar canal alfa

            # Superponer la máscara rotada sobre el frame
            
            frame[y:y+h_masc, x:x+w_masc] = (1.0 - alpha_mask[..., None]) * frame[y:y+h_masc, x:x+w_masc] + alpha_mask[..., None] * overlay_rgb
                    
        # Mostrar el fotograma resultante
        cv2.imshow('Vid', frame)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura y destruye ventanas
vid.release()
cv2.destroyAllWindows()


2024-11-14 15:45:36.593621: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 15:45:36.597797: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 15:45:36.608344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731599136.627183    5361 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731599136.632185    5361 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 15:45:36.650768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins